In [1]:
import sys
import os
import rasterio
import osmnx

import geopandas as gpd
import pandas as pd
import multiprocessing

from shapely.ops import transform
import pyproj

sys.path.append("/home/wb411133/Code/GOSTrocks/src")

import GOSTrocks.ntlMisc as ntlMisc
import GOSTrocks.rasterMisc as rMisc
from GOSTrocks.misc import tPrint

/home/wb411133/.conda/envs/ee/lib/python3.9/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
data_folder = "s3://wbg-geography01/URBANIZATION/MENA/"
ucdb_file       = os.path.join(data_folder,"Extents", "GHS_STAT_UCDB2015MT_GLOBE_R2019A_V1_2.gpkg")
fua_file        = os.path.join(data_folder,"Extents", "GHS_FUA_UCDB2015_GLOBE_R2019A_54009_1K_V1_0.gpkg")
fua_peripheries = os.path.join(data_folder,"Extents", "FUA_peripheries.gpkg")

# Highway features are reclassified to 4 OSMLR classes for simplification and standardization
#   https://mapzen.com/blog/osmlr-2nd-technical-preview/
OSMLR_Classes = {
"motorway":"OSMLR_1",
"motorway_link":"OSMLR_1",
"trunk":"OSMLR_1",
"trunk_link":"OSMLR_1",
"primary":"OSMLR_1",
"primary_link":"OSMLR_1",

"secondary":"OSMLR_2",
"secondary_link":"OSMLR_2",
"tertiary":"OSMLR_2",
"tertiary_link":"OSMLR_2",

"unclassified":"OSMLR_3",
"unclassified_link": "OSMLR_3",
"residential": "OSMLR_3",
"residential_link": "OSMLR_3",

"track": "OSMLR_4",
"service": "OSMLR_4"
}


In [3]:
inF = gpd.read_file(fua_file)
inF = inF.to_crs(4326)
inP = gpd.read_file(fua_peripheries)
if inP.crs != inF.crs:
    inP = inP.to_crs(4326)
inP['geometry'] = inP.buffer(0)

In [4]:
wgs84 = pyproj.CRS('EPSG:4326')
wm = pyproj.CRS('EPSG:3278')

project = pyproj.Transformer.from_crs(wgs84, wm, always_xy=True).transform

def summarize_road_length(curR, core_shp, fua_shp, transformer):
    ''' curR - GeoDataFrame of roads
    core_shp - shapely polygon of core
    transformer - turn wgs84 into 3857
    
    returns - array of length intersecting [core_shp, fua_shp] in metres
    '''
    core_rds = curR.unary_union.intersection(core_shp)
    core_rds = transform(transformer, core_rds)
    fua_rds = curR.unary_union.intersection(fua_shp)
    fua_rds = transform(transformer, fua_rds)    
    return([core_rds.length, fua_rds.length])

In [ ]:
all_res = []
for idx, cur_city in inF.iterrows():
    cur_roads = osmnx.geometries_from_polygon(cur_city['geometry'], {"highway":True})
    cur_roads['OSMLR'] = cur_roads['highway'].map(OSMLR_Classes)
    cur_periphery = inP.loc[inP['eFUA_ID'] == cur_city['eFUA_ID']].iloc[0]
    cur_res = {"eFUA_ID":cur_city['eFUA_ID']}
    for lbl, rds in cur_roads.groupby('OSMLR'):
        rd_length = summarize_road_length(rds, cur_city.geometry, cur_periphery.geometry, project)
        cur_res[f'core_{lbl}_m'] = rd_length[0]
        cur_res[f'per_{lbl}_m'] = rd_length[1]
    all_res.append(cur_res)
    tPrint(idx)

# Multiprocessing

In [5]:
def summarize_road_length(cur_city, cur_p):
    ''' 
    cur_city - geoseries of current city attributes
    cur_p - geoseries of matching periphery
    transformer - turn wgs84 into 3857
    
    returns - array of length intersecting [e_ID, core_shp, fua_shp] in metres
    '''
    wgs84 = pyproj.CRS('EPSG:4326')
    wm = pyproj.CRS('EPSG:3278')
    transformer = pyproj.Transformer.from_crs(wgs84, wm, always_xy=True).transform
    cur_res = {'eFUA_ID':cur_city['eFUA_ID']}
    try:
        try:
            cur_roads = osmnx.geometries_from_polygon(cur_city['geometry'], {"highway":True})
        except:
            cur_roads = osmnx.geometries_from_polygon(cur_city['geometry'].buffer(0), {"highway":True})
        cur_roads['OSMLR'] = cur_roads['highway'].map(OSMLR_Classes)
        for lbl, curR in cur_roads.groupby('OSMLR'):
            core_rds = curR.unary_union.intersection(cur_city.geometry)
            core_rds = transform(transformer, core_rds)
            fua_rds = curR.unary_union.intersection(cur_p.geometry)
            fua_rds = transform(transformer, fua_rds)
            cur_res[f'core_{lbl}_m'] = core_rds.length
            cur_res[f'per_{lbl}_m'] = fua_rds.length
        tPrint(f"Completed {cur_city['eFUA_name']}")
    except:
        tPrint(f"*****Error {cur_city['eFUA_name']}")
    return(cur_res)


In [7]:
# prepare mp arguments
all_args = []
for idx, cur_city in inF.iterrows():
    cur_periphery = inP.loc[inP['eFUA_ID'] == cur_city['eFUA_ID']].iloc[0]
    all_args.append([cur_city, cur_periphery])

In [8]:
summarize_road_length(*all_args[0])

09:05:59	Completed Herat


{'eFUA_ID': 1.0,
 'core_OSMLR_1_m': 312985.9058406732,
 'per_OSMLR_1_m': 51388.56508881661,
 'core_OSMLR_2_m': 781455.7082644084,
 'per_OSMLR_2_m': 308715.4084912754,
 'core_OSMLR_3_m': 4039326.8120645066,
 'per_OSMLR_3_m': 1565801.8344054415,
 'core_OSMLR_4_m': 433957.4827416821,
 'per_OSMLR_4_m': 265087.25905505934}

In [9]:
with multiprocessing.Pool(70) as pool:
    res = pool.starmap(summarize_road_length, all_args)

/home/wb411133/.conda/envs/ee/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


09:06:26	Completed Herat
09:07:24	Completed Tamacine
09:07:24	Completed Ksar El Boukhari
09:07:24	Completed Gaoua
09:07:25	Completed Shamakhi
09:07:25	Completed Janaúba
09:07:25	Completed Paranaguá
09:07:26	Completed Elbasan
09:07:26	Completed Itapoã
09:07:26	Completed Daozhen
09:07:26	Completed Bambari
09:07:27	Completed Foumban
09:07:27	Completed Hanzhong
09:07:28	Completed Campos dos Goytacazes
09:07:28	Completed Santarém
09:07:28	Completed Changli
09:07:28	Completed Xinjin
09:07:28	Completed Bagé
09:07:29	Completed Yangxian
09:07:29	Completed Jinchang
09:07:29	Completed Hulunbuir
09:07:29	Completed Sylhet
09:07:29	Completed Barreiras
09:07:29	Completed Gongnong
09:07:29	Completed Hohhot
09:07:29	Completed Guzarah
09:07:30	Completed Yining/Qulja
09:07:30	Completed Ngozi
09:07:31	Completed M'banza Congo
09:07:31	Completed Iquique
09:07:31	Completed Barkhada
09:07:32	Completed Anápolis
09:07:32	Completed Guiyang
09:07:33	Completed Aracaju
09:07:33	Completed Resistencia
09:07:38	Comple

TopologyException: Input geom 1 is invalid: Ring Self-intersection at 91.906892885759774 22.504286228991546


09:11:44	*****Error Chattogram
09:11:45	Completed Tailândia
09:11:45	Completed Uruguaiana
09:11:45	Completed Marília
09:11:45	Completed Santa Cruz do Capibaribe
09:11:48	Completed Mbalmayo
09:11:48	Completed La Serena
09:11:50	Completed Alfenas
09:11:59	Completed Mostar
09:11:59	Completed Ain Oussara
09:12:01	Completed La Rioja
09:12:04	Completed Liège
09:12:06	Completed Goya
09:12:29	Completed Anyang
09:12:29	Completed 石横镇
09:12:29	Completed Weixin
09:12:29	Completed Linfen
09:12:30	Completed Chengcheng
09:12:30	Completed Wenshan
09:12:30	Completed Guiyang
09:12:30	Completed Qianjiang
09:12:30	Completed Ruicheng
09:12:31	Completed Shancheng
09:12:31	Completed Chaoyang
09:12:31	Completed Shanghe
09:12:32	Completed Ning'an
09:12:32	Completed Guoyang
09:12:33	Completed Macheng
09:12:33	Completed Masallı
09:12:33	Completed Yuanqu
09:12:33	Completed Ruichang
09:12:34	Completed Huanghua
09:12:34	Completed Ganyu
09:12:34	Completed Taihe
09:12:34	Completed 益林镇
09:12:34	Completed Xiangyin
09:1

TopologyException: Input geom 1 is invalid: Ring Self-intersection at 112.42268958507427 34.788780015169721


...

12:45:08	*****Error Palm Bay
12:46:23	Completed Nashua
12:47:15	Completed Bradenton
12:47:38	Completed Lowell
12:49:01	Completed Manchester
12:49:17	Completed South Bend
12:50:24	Completed Lawrence
12:50:29	Completed Jacksonville
12:50:36	Completed Appleton
12:51:36	Completed Palm Coast
12:51:44	Completed Portland
12:52:04	Completed Charleston
12:52:46	Completed Concordia
12:52:52	Completed Daytona Beach
12:53:48	Completed Paysandú
12:53:51	Completed Tampa
12:54:01	Completed La Crosse
12:54:50	Completed Rivera
12:55:13	Completed Dallas
12:56:03	Completed Montevideo
12:56:21	Completed Lawton
12:57:06	Completed Maldonado
12:57:10	Completed Dayton


TopologyException: Input geom 1 is invalid: Ring Self-intersection at -76.715044861323534 39.139578396112682


12:57:11	*****Error Washington D.C.
12:57:28	Completed Billings
12:58:08	Completed Khujayli
12:58:30	Completed Hagerstown
12:58:38	Completed Port Arthur
12:59:11	Completed Nukus
12:59:49	Completed Frederick
13:00:13	Completed Khiva
13:00:59	Completed State College
13:04:06	Completed Charlotte
13:06:14	Completed Oklahoma City


TopologyException: Input geom 1 is invalid: Ring Self-intersection at -76.715044861323534 39.139578396112682


13:07:05	*****Error Baltimore
13:07:24	Completed Longview
13:08:30	Completed Lake Charles
13:08:54	Completed Fort Wayne
13:09:43	Completed Shreveport
13:10:07	Completed Green Bay
13:11:16	Completed Holland
13:12:02	Completed Harrisburg
13:12:36	Completed Kalamazoo
13:13:20	Completed York
13:14:14	Completed Wichita
13:17:21	Completed Grand Rapids
13:18:23	Completed Buffalo
13:19:50	Completed Lancaster
13:28:50	Completed Columbus
13:30:25	Completed Winston-Salem
13:37:58	Completed Philadelphia
13:39:26	Completed Reading
13:40:45	Completed West Chester
13:45:28	Completed Rochester


In [10]:
osm_res = pd.DataFrame(res)
bad_fua = osm_res.loc[osm_res['core_OSMLR_2_m'].isna() * osm_res['core_OSMLR_3_m'].isna(),'eFUA_ID'].values
updatedF = inF.copy()
updatedF.set_index("eFUA_ID")
all_res = []
for cur_fua in bad_fua:
    cur_city = inF.loc[inF['eFUA_ID'] == cur_fua].iloc[0]
    cur_periphery = inP.loc[inP['eFUA_ID'] == cur_city['eFUA_ID']].iloc[0]
    try:
        cur_city['geometry'] = cur_city['geometry'].buffer(0)
        cur_periphery['geometry'] = cur_periphery['geometry'].buffer(0)
        cur_res = summarize_road_length(cur_city, cur_periphery)
        all_res.append(cur_res)
    except:
        tPrint("")

/home/wb411133/.conda/envs/ee/lib/python3.9/site-packages/pandas/core/computation/expressions.py:204: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  warnings.warn(


13:45:31	
13:45:31	
13:45:31	
13:45:31	
13:45:31	
13:45:31	
13:45:31	
13:45:31	
13:45:31	
13:45:31	
13:46:02	Completed Mumbai
13:46:02	
13:46:02	
13:46:02	
13:46:02	
13:46:02	
13:47:03	Completed Pandaria
13:48:04	Completed UNNAMED
13:49:05	Completed Bausi
13:49:05	
13:49:24	Completed Matsuyama
14:23:11	Completed Tokyo
14:24:26	*****Error UNNAMED
14:24:26	
14:26:00	Completed Amsterdam
14:26:01	
14:26:01	
14:27:02	Completed Mahuta
14:28:03	Completed Ungwan Bahagu
14:28:03	
14:28:03	
14:28:03	
14:28:03	


In [13]:
fixed_roads = pd.DataFrame(all_res)
fixed_roads = fixed_roads.set_index('eFUA_ID')
final_dataset = osm_res.set_index('eFUA_ID')
final_dataset.update(fixed_roads)
final_dataset

/home/wb411133/.conda/envs/ee/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,core_OSMLR_1_m,per_OSMLR_1_m,core_OSMLR_2_m,per_OSMLR_2_m,core_OSMLR_3_m,per_OSMLR_3_m,core_OSMLR_4_m,per_OSMLR_4_m
eFUA_ID,,,,,,,,
1.0,312985.905841,5.138857e+04,781455.708264,3.087154e+05,4.039327e+06,1.565802e+06,433957.482742,265087.259055
192.0,75072.264156,2.088360e+04,90789.961547,4.629662e+04,8.559492e+05,4.103882e+05,115968.663338,42009.041709
354.0,NaN,NaN,72881.046342,1.178040e-08,1.757049e+05,6.446954e-08,NaN,NaN
505.0,23526.426134,0.000000e+00,32848.409563,0.000000e+00,6.984145e+04,0.000000e+00,4231.993902,0.000000
648.0,72686.230616,1.727627e+04,161581.966915,6.567590e+04,6.167274e+05,1.840791e+05,387.948803,0.000000
...,...,...,...,...,...,...,...,...
1739.0,4082.129274,1.678966e-09,463.764289,0.000000e+00,2.494004e+04,0.000000e+00,1130.613141,0.000000
1908.0,NaN,NaN,3394.678536,0.000000e+00,5.615943e+03,0.000000e+00,4532.037366,0.000000
2062.0,13967.250525,7.746106e+03,42726.055883,1.525569e+04,2.026332e+05,6.814802e+04,35186.777619,22518.626823


In [14]:
final_dataset.to_csv(os.path.join(data_folder, "ZONAL_RES", "OSM_ROAD_LENGTH", "OSMLR_Classes_length_m.csv"))

/home/wb411133/.conda/envs/ee/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/wb411133/.conda/envs/ee/lib/python3.9/site-packages/pandas/compat/_optional.py:116: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if distutils.version.LooseVersion(version) < minimum_version:
/home/wb411133/.conda/envs/ee/lib/python3.9/site-packages/setuptools/_distutils/version.py:345: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/home/wb411133/.conda/envs/ee/lib/python3.9/site-packages/fsspec/registry.py:231: DeprecationWarning: distutils Version classes are deprecated. Use packaging.ver

In [15]:
data_folder

's3://wbg-geography01/URBANIZATION/MENA/'

In [ ]:

data_folder